# Question 1
What are some ways you could find out if a python object is an iterable?












#try iterating through it!


In [1]:
try:
    obj = None
    test = [_ for x in obj]
except TypeError:
    print("not iterable")

not iterable


# Question 2 
You are given a function, `primitiveHash()`, that accepts either a string or a number, and outputs its hash. 

<br>
Implement a function `tupleHash()` that computes the hash of a tuple. The tuple can contain strings, numbers, or other tuples. 

<br>

You are not allowed to use the built-in `hash()` method (note that `primitiveHash()` uses it).
<br>

This question is somewhat open-ended - there is more than one possible implementation for this function. 
This hash function should be designed such that it could be used by a dictionary implementation.
<br>

For extra context, view a reference for what is considered hashable in python here:
<br>
https://stackoverflow.com/a/14535739

Dictionary keys must be hashable items. Hashable items must be immutable and comparable. Mutable objects (lists/dictionaries) are not hashable because their hash can change over the lifetime of the object.


In [21]:
# Execute this cell but don't touch this function
def primitiveHash(arg):
    if type(arg) in [int,str,float]:
        return hash(arg)
    raise Exception("primitiveHash only accepts a number or a string")


In [43]:
def tupleHash(my_tuple):
    return compute_hash(my_tuple, 0)

def compute_hash(my_tuple, tuple_hash):
    #tuple contains strings, numbers, or other tuples
    for item in my_tuple:
        if type(item) in [int, float,str]:
            tuple_hash += primitiveHash(item)
        elif type(item) == tuple:
            tuple_hash += compute_hash(item, tuple_hash)
            
    return int(tuple_hash)
            
tupleHash((4, 98, "hello", "hfhdu", (5, 3, 5), 8678976, 9802.54, "zzzzzz", "98ihj  soihj"))

8601283112370080271

In [44]:
print(hash("a"), hash("b"), hash("c"))
print(hash(("a","b","c")))

5146297991148823325 -3260379370662778778 -6256434123508558075
-3726622698212719136


# Question 3
Implement the functionality of the python set API, such that this code behaves as expected. 
<br>
You may not use `set()`. You probably shouldn't be using `list.append`.

In [151]:
class myset:
    # Implement
    def __init__(self, iter_item=None):
        if not hasattr(iter_item, '__iter__'):
            raise TypeError(iter_item + " not iterable")
        self.myset = {}
        
        if iter_item is not None:
            for item in iter_item:
                self.add(item)
              
    def add(self, ele):   
        self.myset[ele] = True
            
    def remove(self, ele):
        self.myset.pop(ele, None)
        
        
    def __str__(self):
        keys = list(self.myset.keys())
        return ",".join(str(v) for v in keys)
    
    def __iter__(self):
        return myset_iterator(self)
    
class myset_iterator:
    def __init__(self, myset):
        self._set = myset.myset
        self._keys = list(myset.myset.keys())
        
    def __next__(self): 
        if self._index < len(self._set):
            result = self._keys[self._index]    
            self._index += 1
            return result
        
        raise StopIteration

In [154]:

        
# Execute this cell to test your implementation (but do not edit)
def question3():
    x = myset([1, 2, 4, 5, 10, 54, 32, 10, 22])  # make a set
    print(x)  # {1,2,3,5}

    2 in x  # True

    try:
        myset(2)  # error: 'int' object is not iterable
    except Exception:
        print(Exception)

    x.add(8)  # adds 8 to the set
    x.add(3)  # no-op since 3 is already present
    x.remove(2)  # removes 2

    print(x)


question3()

1,2,4,5,10,54,32,22
<class 'Exception'>
1,4,5,10,54,32,22,8,3


# Question 4
Implement the functionality of the python dict API, such that this code behaves as expected.
<br>
You may not use `dict()`.

In [91]:
class Node:
    def __init__(self, val, key):
        self.val = val
        self.key = key
        self.next = None
        
class LinkedList:
    def __init__(self):
        self.head = None
        
    def push(self, val, key):
        new_node = Node(val, key)
        new_node.next = self.head
        self.head = new_node
        
    def delete_node(self, key):
        temp = self.head
        if temp:
            if temp.key == key:
                self.head = temp.next
                temp = None
                return
        
        while temp:
            if temp.key == key:
                break
            prev = temp
            temp = temp.next
            
        if not temp:
            return
        
        prev.next = temp.next
        temp = None

class mydict:
    def __init__(self):
        self.capacity = 32
        self.arr = [None for _ in range(self.capacity)]
        self.num_elements = 0
        self.load_factor = .75
        self.visited = set()
        
    def __getitem__(self,item):
        item_index = hash(item) % self.capacity
        ll = self.arr[item_index]
        node = ll.head
        while node:
            if node.key == item:
                return node.val
            node = node.next
            
        raise KeyError
        
    def __setitem__(self, item, val):
        if self.num_elements / self.capacity >= self.load_factor:
            self._copy_and_allocate()
        
        item_index = hash(item) % self.capacity
        ll_head = self.arr[item_index]
        
        if not ll_head:
            ll = LinkedList()
            self.arr[item_index] = ll
        else:
            head = ll_head.head
            while head:
                if head.key == item:
                    head.val = val
                    return
                head = head.next
            
        self.arr[item_index].push(val, item)
        self.num_elements += 1
        
            
    def _copy_and_allocate(self):
        self.capacity *= 2
        copy = [None for _ in range(self.capacity)]
        for item in self.arr:
            while item:
                new_node = Node(item.val, item.key)
                item_index = hash(item.key) % self.capacity
                
                if copy[item_index]:
                    new_node.next = copy[item_index]
                else:
                    copy[item_index] = new_node
                
                item = item.next
                
        self.arr = copy
        
            
    def __delitem__(self, item):
        item_index = hash(item) % self.capacity
        head = self.arr[item_index]
        if head:
            head.delete_node(item)
            
        
    def __str__(self):
        visited = set()
        visited.add(id(self))
        dict_str = "{"
        return self._helper(visited, dict_str)
    
    
    def _helper(self, visited, dict_str):
        for key in self:
            value = self[key]
            if id(value) not in visited:
                #print(type(value) == mydict)
                if type(value) == mydict:
                    visited.add(id(value))
                kv_str = str(key) + ":" + (str(value)) + ", "    
                # need to handle case "key: {...}" if val is already visited
                #dict_str += kv_str
            else:
                kv_str = str(key) + ":" + "{...}" + ", "
                
            dict_str += kv_str
        
        return dict_str[:-2] + '}'        

    
    def __iter__(self):
        return MyDictIter(self)
    
    def values(self):
        for item in self.arr:
            if item is not None:
                head = item.head
                while head:
                    yield head.val
                    head = head.next
                    
class MyDictIter:
    def __init__(self, mydict):
        self.index = 0
        self.node = None
        self.mydict = mydict
        
    def __next__(self):
        while self.index < self.mydict.capacity and not self.node:
            ll = self.mydict.arr[self.index]
            if ll:
                self.node = ll.head
            self.index += 1
            
        if self.index == self.mydict.capacity: 
            raise StopIteration
            
        result = ll.head.key
        self.node = ll.head.next
        return result




In [92]:
# Execute this cell to test your implementation (but do not edit)
def question4():
    x = mydict()
    x[1] = "foo"
    #print(x)
    x["bar"] = 2
    #print(x)
    x["aaaa"] = 35
    x[(1, 2, 3)] = x
    
    print(x)
    del x[1]
#     print(x)
    #print(x)  # {"bar": 2, (1,2,3): {...}}
    

    x["bar"] = 3
    print("\nPrinting keys:")
    for key in x:
        print(key)  # prints each key ("bar", (1,2,3))

    print("\nPrinting values:")
    for value in x.values():
        print(value)  # prints each value (3, {"bar": 3, (1,2,3): {...}})

    
question4()

{1:foo, (1, 2, 3):{...}, bar:2, aaaa:35}

Printing keys:
(1, 2, 3)
bar
aaaa

Printing values:
{(1, 2, 3):{...}, bar:3, aaaa:35}
3
35


# Question 5
What does the `dir()` function do?

returns all properties and methods of an object

# Question 6
Let's say you have a class, `Goose`.

```
class Goose:
   def __init__(self,quack):
       self.quack = quack
```
You would like to be able to use the `+`, `*`, `/`, and `-` operators with instances of `Goose`, that operates on their `quack` values. <br>
Example:<br>
```
goose1 = Goose(2)
goose2 = Goose(3)
goose3 = goose1 + goose2
print(goose3.quack) # prints 5
```

How would you implement the `Goose` class to support this?

In [39]:
# Implement
class Goose:
    def __init__(self,quack):
        self.quack = quack
        
    def __add__(self, other):
        return Goose(self.quack + other.quack)
    
    def __mul__(self, other):
        return Goose(self.quack * other.quack)
    
    def __truediv__(self, other):
        return Goose(self.quack / other.quack)

    def __sub__(self, other):
        return Goose(self.quack - other.quack)
    

In [40]:
# Execute this cell to test your implementation (but do not edit)
g1 = Goose(12)
g2 = Goose(3)
g3 = g1 + g2
g4 = g1 * g2 
g5 = g1 / g2
g6 = g1 - g2
assert(g3.quack == 15)
assert(g4.quack == 36)
assert(g5.quack == 4)
assert(g6.quack == 9)

# Question 7
Given this modified base implementation of `Goose`:
```
class Goose:
    def __init__(self,quack):
        if type(quack) != int or quack < 1:
            raise Exception("Need positive integer quack")
        self.quack = quack
```
Implement `Goose` to support the following behavior:
```
goose = Goose(5)
for thing in goose:
    print(thing)
```
Output:
```
5
4
3
2
1
```

In [41]:
# Implement
class Goose:
    def __init__(self,quack):
        if type(quack) != int or quack < 1:
            raise Exception("Need positive integer quack")
        self.quack = quack
        
    def __iter__(self):
        return GooseIter(self)
    
class GooseIter:
    def __init__(self, goose):
        self._index = 0
        self._range = range(1, goose.quack + 1)
        
    def __next__(self):
        if self._index < len(self._range):
            result = self._range[self._index]
            self._index += 1
            return result
        
        raise StopIteration
        

In [42]:
goose = Goose(5)
for thing in goose:
    print(thing)

1
2
3
4
5


# Question 8
Given the same implementation of `Goose` as from question 7, and the following code:
```
goose = Goose(5)
for thing in goose:
    pass
```

Implement `Goose` such that the following output is produced instead
Output:
```
5 quack-quack-quack-quack-quack
4 quack-quack-quack-quack
3 quack-quack-quack
2 quack-quack
1 quack
```

In [43]:
# Implement
class Goose:
    def __init__(self,quack):
        if type(quack) != int or quack < 1:
            raise Exception("Need positive integer quack")
        self.quack = quack
        
    def __iter__(self):
        return GooseIter(self)
    
class GooseIter:
    def __init__(self, goose):       
        self._range = range(1, goose.quack + 1)
        self._index = len(self._range) - 1
        
    def __next__(self):
        if self._index >= 0:
            quack_str = '-'.join(['quack'] * (self._range[self._index]))
            result = str(self._range[self._index]) + ' ' + quack_str
            self._index -= 1
            return result
        
        raise StopIteration

In [44]:
goose = Goose(5)
for thing in goose:
    print(thing)

5 quack-quack-quack-quack-quack
4 quack-quack-quack-quack
3 quack-quack-quack
2 quack-quack
1 quack
